In [1]:
import os
import sys
sys.path.insert(0,'..')

from bidst.transformers import NUCorrectionTransformer
from bidst.transformers import SkullStrippingTransformer
from bidst.transformers import TissueSegmentation
from bidst.transformers import StatAggregator
from bidst.pipeline import BIDSPipeline

from sklearn.linear_model import LogisticRegression
import numpy as np

In [2]:
import nibabel as nib
from functools import reduce
from operator import mul

def get_tissue_stats(subject_folder):
    
    ext_dict = {}
    ext_dict['WM_volume'] = 'WM_probtissue'
    ext_dict['GM_volume'] = 'GM_probtissue'
    ext_dict['CSF_volume'] = 'CSF_probtissue'
    
    files_list = os.listdir(subject_folder)
    
    results = {}
    for key in ext_dict.keys():
        cur_file = [item for item in files_list if ext_dict[key] in item]
        if len(cur_file) > 1:
            raise ValueError('Several candiadates in the folder')
        else:
            cur_file = cur_file[0]
        cur_img = nib.load(subject_folder + '/' + cur_file)
        voxel_sum = cur_img.get_data()
        voxel_sum = (voxel_sum >= 0.5).sum()
        voxel_vol = reduce(mul, cur_img.header.get_zooms())
        all_vol = voxel_sum * voxel_vol
        results[key] = all_vol
    
    return results

In [3]:
! rm -r ../bidst/tests/data/ds114/derivatives/

rm: cannot remove '../bidst/tests/data/ds114/derivatives/': No such file or directory


In [4]:
! tree -a ../bidst/tests/data/ds114/

../bidst/tests/data/ds114/
├── sub-01
│   ├── ses-retest
│   │   └── anat
│   │       └── sub-01_ses-retest_T1w.nii.gz
│   └── ses-test
│       └── anat
│           └── sub-01_ses-test_T1w.nii.gz
├── sub-02
│   ├── ses-retest
│   │   └── anat
│   │       └── sub-02_ses-retest_T1w.nii.gz
│   └── ses-test
│       └── anat
│           └── sub-02_ses-test_T1w.nii.gz
├── sub-03
│   ├── ses-retest
│   │   └── anat
│   │       └── sub-03_ses-retest_T1w.nii.gz
│   └── ses-test
│       └── anat
│           └── sub-03_ses-test_T1w.nii.gz
└── sub-04
    ├── ses-retest
    │   └── anat
    │       └── sub-04_ses-retest_T1w.nii.gz
    └── ses-test
        └── anat
            └── sub-04_ses-test_T1w.nii.gz

20 directories, 8 files


In [5]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'example_pipeline'

nucorrection = NUCorrectionTransformer(gather_steps=('source', dict(extensions='T1w.nii.gz')))

skullstripping = SkullStrippingTransformer(gather_steps=('nucorrection', dict(extensions='nu.nii.gz')))

tissuesegmentation = TissueSegmentation(gather_steps=('skullstripping', dict(extensions='brain.nii.gz')))

aggregator = StatAggregator(function=get_tissue_stats, 
                            gather_steps=('tissuesegmentation', dict(extensions='probtissue.nii.gz')))

IDS = ['01']

pipe = BIDSPipeline(steps=[('nucorrection', nucorrection),
                           ('skullstripping', skullstripping),
                           ('tissuesegmentation', tissuesegmentation),
                           ('aggregator', aggregator)],
             pipeline_name=pipeline_name, project_path=project_path)

In [6]:
pipe.fit_transform(IDS)

/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/example_pipeline/steps/nucorrection/sub-01/ses-retest/anat/sub-01_ses-retest_T1w_nu.nii.gz
/work/soft/BIDSTransformers/bidst/tests/data/ds114/sub-01/ses-retest/anat/sub-01_ses-retest_T1w.nii.gz
170901-20:55:32,344 interface INFO:
	 stdout 2017-09-01T20:55:32.343976:
170901-20:55:32,345 interface INFO:
	 stdout 2017-09-01T20:55:32.343976:Running N4 for 3-dimensional images.
170901-20:55:32,345 interface INFO:
	 stdout 2017-09-01T20:55:32.343976:
170901-20:55:32,346 interface INFO:
	 stdout 2017-09-01T20:55:32.343976:Mask not read.  Creating Otsu mask.
170901-20:55:32,847 interface INFO:
	 stdout 2017-09-01T20:55:32.847784:Current level = 1
170901-20:55:32,849 interface INFO:
	 stdout 2017-09-01T20:55:32.847784:  Iteration 1 (of 50).   Current convergence value = 0.000264527 (threshold = 0)
170901-20:55:32,849 interface INFO:
	 stdout 2017-09-01T20:55:32.847784:  Iteration 2 (of 50).   Current convergence value = 0.000262299 

170901-20:55:37,399 interface INFO:
	 stdout 2017-09-01T20:55:37.394611:  Iteration 50 (of 50).   Current convergence value = 0.000193532 (threshold = 0)
170901-20:55:37,399 interface INFO:
	 stdout 2017-09-01T20:55:37.394611:Current level = 2
170901-20:55:37,400 interface INFO:
	 stdout 2017-09-01T20:55:37.394611:  Iteration 1 (of 50).   Current convergence value = 0.000425829 (threshold = 0)
170901-20:55:37,901 interface INFO:
	 stdout 2017-09-01T20:55:37.901890:  Iteration 2 (of 50).   Current convergence value = 0.000419074 (threshold = 0)
170901-20:55:37,902 interface INFO:
	 stdout 2017-09-01T20:55:37.901890:  Iteration 3 (of 50).   Current convergence value = 0.000411931 (threshold = 0)
170901-20:55:37,903 interface INFO:
	 stdout 2017-09-01T20:55:37.901890:  Iteration 4 (of 50).   Current convergence value = 0.000406177 (threshold = 0)
170901-20:55:37,904 interface INFO:
	 stdout 2017-09-01T20:55:37.901890:  Iteration 5 (of 50).   Current convergence value = 0.000400012 (thresh

170901-20:55:42,451 interface INFO:
	 stdout 2017-09-01T20:55:42.447924:  Iteration 3 (of 50).   Current convergence value = 0.000954957 (threshold = 0)
170901-20:55:42,953 interface INFO:
	 stdout 2017-09-01T20:55:42.953058:  Iteration 4 (of 50).   Current convergence value = 0.000926239 (threshold = 0)
170901-20:55:42,954 interface INFO:
	 stdout 2017-09-01T20:55:42.953058:  Iteration 5 (of 50).   Current convergence value = 0.000895458 (threshold = 0)
170901-20:55:42,954 interface INFO:
	 stdout 2017-09-01T20:55:42.953058:  Iteration 6 (of 50).   Current convergence value = 0.000869874 (threshold = 0)
170901-20:55:42,955 interface INFO:
	 stdout 2017-09-01T20:55:42.953058:  Iteration 7 (of 50).   Current convergence value = 0.000843455 (threshold = 0)
170901-20:55:42,956 interface INFO:
	 stdout 2017-09-01T20:55:42.953058:  Iteration 8 (of 50).   Current convergence value = 0.000818881 (threshold = 0)
170901-20:55:43,457 interface INFO:
	 stdout 2017-09-01T20:55:43.457577:  Iteratio

170901-20:55:48,3 interface INFO:
	 stdout 2017-09-01T20:55:48.002328:  Iteration 6 (of 50).   Current convergence value = 0.00116415 (threshold = 0)
170901-20:55:48,3 interface INFO:
	 stdout 2017-09-01T20:55:48.002328:  Iteration 7 (of 50).   Current convergence value = 0.00111118 (threshold = 0)
170901-20:55:48,4 interface INFO:
	 stdout 2017-09-01T20:55:48.002328:  Iteration 8 (of 50).   Current convergence value = 0.0010646 (threshold = 0)
170901-20:55:48,5 interface INFO:
	 stdout 2017-09-01T20:55:48.002328:  Iteration 9 (of 50).   Current convergence value = 0.00102261 (threshold = 0)
170901-20:55:48,506 interface INFO:
	 stdout 2017-09-01T20:55:48.506631:  Iteration 10 (of 50).   Current convergence value = 0.000982827 (threshold = 0)
170901-20:55:48,507 interface INFO:
	 stdout 2017-09-01T20:55:48.506631:  Iteration 11 (of 50).   Current convergence value = 0.000946726 (threshold = 0)
170901-20:55:48,508 interface INFO:
	 stdout 2017-09-01T20:55:48.506631:  Iteration 12 (of 50

170901-20:55:52,56 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:       1: _1 (0x3604b70)
170901-20:55:52,56 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:     Required Input Names: Primary
170901-20:55:52,57 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:     NumberOfRequiredInputs: 1
170901-20:55:52,61 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:     Outputs: 
170901-20:55:52,62 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:       Primary: (0x35f9f90)
170901-20:55:52,63 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:     Indexed Outputs: 
170901-20:55:52,65 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:       0: Primary (0x35f9f90)
170901-20:55:52,66 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:     NumberOfRequiredOutputs: 1
170901-20:55:52,67 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:     Number Of Threads: 1
170901-20:55:52,68 interface INFO:
	 stdout 2017-09-01T20:55:52.041153:     ReleaseDataFlag: Off
170901-20:55:52,69 

...

170901-20:56:05,368 interface INFO:
	 stdout 2017-09-01T20:56:05.365360:  Iteration 39 (of 50).   Current convergence value = 0.000255441 (threshold = 0)
170901-20:56:05,368 interface INFO:
	 stdout 2017-09-01T20:56:05.365360:  Iteration 40 (of 50).   Current convergence value = 0.000252225 (threshold = 0)
170901-20:56:05,369 interface INFO:
	 stdout 2017-09-01T20:56:05.365360:  Iteration 41 (of 50).   Current convergence value = 0.000249309 (threshold = 0)
170901-20:56:05,870 interface INFO:
	 stdout 2017-09-01T20:56:05.870897:  Iteration 42 (of 50).   Current convergence value = 0.000246471 (threshold = 0)
170901-20:56:05,871 interface INFO:
	 stdout 2017-09-01T20:56:05.870897:  Iteration 43 (of 50).   Current convergence value = 0.000243748 (threshold = 0)
170901-20:56:05,872 interface INFO:
	 stdout 2017-09-01T20:56:05.870897:  Iteration 44 (of 50).   Current convergence value = 0.000240966 (threshold = 0)
170901-20:56:05,873 interface INFO:
	 stdout 2017-09-01T20:56:05.870897:  It

170901-20:56:10,428 interface INFO:
	 stdout 2017-09-01T20:56:10.427159:  Iteration 42 (of 50).   Current convergence value = 0.000482016 (threshold = 0)
170901-20:56:10,428 interface INFO:
	 stdout 2017-09-01T20:56:10.427159:  Iteration 43 (of 50).   Current convergence value = 0.000473896 (threshold = 0)
170901-20:56:10,429 interface INFO:
	 stdout 2017-09-01T20:56:10.427159:  Iteration 44 (of 50).   Current convergence value = 0.000465534 (threshold = 0)
170901-20:56:10,430 interface INFO:
	 stdout 2017-09-01T20:56:10.427159:  Iteration 45 (of 50).   Current convergence value = 0.000456484 (threshold = 0)
170901-20:56:10,430 interface INFO:
	 stdout 2017-09-01T20:56:10.427159:  Iteration 46 (of 50).   Current convergence value = 0.000450841 (threshold = 0)
170901-20:56:10,932 interface INFO:
	 stdout 2017-09-01T20:56:10.932181:  Iteration 47 (of 50).   Current convergence value = 0.000441746 (threshold = 0)
170901-20:56:10,933 interface INFO:
	 stdout 2017-09-01T20:56:10.932181:  It

170901-20:56:14,975 interface INFO:
	 stdout 2017-09-01T20:56:14.972352:  Iteration 45 (of 50).   Current convergence value = 0.000516817 (threshold = 0)
170901-20:56:15,476 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:  Iteration 46 (of 50).   Current convergence value = 0.000512361 (threshold = 0)
170901-20:56:15,477 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:  Iteration 47 (of 50).   Current convergence value = 0.000507059 (threshold = 0)
170901-20:56:15,478 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:  Iteration 48 (of 50).   Current convergence value = 0.000503925 (threshold = 0)
170901-20:56:15,479 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:  Iteration 49 (of 50).   Current convergence value = 0.000499756 (threshold = 0)
170901-20:56:15,480 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:  Iteration 50 (of 50).   Current convergence value = 0.000496092 (threshold = 0)
170901-20:56:15,481 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:   N

170901-20:56:15,554 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:         BufferedRegion: 
170901-20:56:15,555 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:           Dimension: 3
170901-20:56:15,556 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:           Index: [0, 0, 0]
170901-20:56:15,557 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:           Size: [11, 11, 11]
170901-20:56:15,557 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:         RequestedRegion: 
170901-20:56:15,558 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:           Dimension: 3
170901-20:56:15,559 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:           Index: [0, 0, 0]
170901-20:56:15,561 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:           Size: [11, 11, 11]
170901-20:56:15,562 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:         Spacing: [31.5, 24.6881, 31.5]
170901-20:56:15,564 interface INFO:
	 stdout 2017-09-01T20:56:15.476392:         Origin: [30.88

CSF_volume      GM_volume      WM_volume
01 retest  239685.182632  417931.285227  381171.795038
   test    276922.990036  425446.851959  392663.598212

In [7]:
!tree -a ../bidst/tests/data/ds114/

../bidst/tests/data/ds114/
├── derivatives
│   └── example_pipeline
│       └── steps
│           ├── nucorrection
│           │   └── sub-01
│           │       ├── ses-retest
│           │       │   └── anat
│           │       │       └── sub-01_ses-retest_T1w_nu.nii.gz
│           │       └── ses-test
│           │           └── anat
│           │               └── sub-01_ses-test_T1w_nu.nii.gz
│           ├── skullstripping
│           │   └── sub-01
│           │       ├── ses-retest
│           │       │   └── anat
│           │       │       └── sub-01_ses-retest_T1w_brain.nii.gz
│           │       └── ses-test
│           │           └── anat
│           │               └── sub-01_ses-test_T1w_brain.nii.gz
│           └── tissuesegmentation
│               └── sub-01
│                   ├── ses-retest
│                   │   └── anat
│                   │       ├── sub-01_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
│                   │       ├── sub-01_ses-retest_T1w_br

In [8]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'test_pipeline1'

skullstripping = SkullStrippingTransformer(gather_steps=('source', dict(extensions='T1w.nii.gz')))

tissuesegmentation = TissueSegmentation(gather_steps=('skullstripping', dict(extensions='T1w_brain.nii.gz')))

aggregator = StatAggregator(function=get_tissue_stats,
                            gather_steps=('tissuesegmentation', dict(extensions='WM_probtissue.nii.gz')))

IDS = ['02']


pipe = BIDSPipeline(steps=[('skullstripping', skullstripping),
                           ('tissuesegmentation', tissuesegmentation),
                           ('aggregator', aggregator)],
             pipeline_name=pipeline_name, project_path=project_path)

In [9]:
pipe.fit_transform(IDS)

/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline1/steps/skullstripping/sub-02/ses-retest/anat/sub-02_ses-retest_T1w_brain.nii.gz
/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline1/steps/skullstripping/sub-02/ses-test/anat/sub-02_ses-test_T1w_brain.nii.gz
Renaming filename: sub-02_ses-retest_T1w_brain_mixeltype.nii.gz
New filename: sub-02_ses-retest_T1w_brain_mixeltype.nii.gz
Renaming filename: sub-02_ses-retest_T1w_brain_pve_1.nii.gz
New filename: sub-02_ses-retest_T1w_brain_class-GM_probtissue.nii.gz
Renaming filename: sub-02_ses-retest_T1w_brain_pve_2.nii.gz
New filename: sub-02_ses-retest_T1w_brain_class-WM_probtissue.nii.gz
Renaming filename: sub-02_ses-retest_T1w_brain_pve_0.nii.gz
New filename: sub-02_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
Renaming filename: sub-02_ses-retest_T1w_brain_pveseg.nii.gz
New filename: sub-02_ses-retest_T1w_brain_pveseg.nii.gz
Renaming filename: sub-02_ses-retest_T1w_brain_seg.nii.gz
New 

CSF_volume      GM_volume      WM_volume
02 retest  247122.684898  449870.986028  468183.542938
   test    235231.048205  411737.814129  440105.344166

In [10]:
!tree -a ../bidst/tests/data/ds114/

../bidst/tests/data/ds114/
├── derivatives
│   ├── example_pipeline
│   │   └── steps
│   │       ├── nucorrection
│   │       │   └── sub-01
│   │       │       ├── ses-retest
│   │       │       │   └── anat
│   │       │       │       └── sub-01_ses-retest_T1w_nu.nii.gz
│   │       │       └── ses-test
│   │       │           └── anat
│   │       │               └── sub-01_ses-test_T1w_nu.nii.gz
│   │       ├── skullstripping
│   │       │   └── sub-01
│   │       │       ├── ses-retest
│   │       │       │   └── anat
│   │       │       │       └── sub-01_ses-retest_T1w_brain.nii.gz
│   │       │       └── ses-test
│   │       │           └── anat
│   │       │               └── sub-01_ses-test_T1w_brain.nii.gz
│   │       └── tissuesegmentation
│   │           └── sub-01
│   │               ├── ses-retest
│   │               │   └── anat
│   │               │       ├── sub-01_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
│   │               │       ├── sub-01_ses-retest_T1w_br

In [11]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'test_pipeline2'

nucorrection = NUCorrectionTransformer(gather_steps=('source', dict(extensions='T1w.nii.gz')))

tissuesegmentation = TissueSegmentation(gather_steps=('nucorrection', dict(extensions='nu.nii.gz')))

aggregator = StatAggregator(function=get_tissue_stats,
                            gather_steps=('tissuesegmentation', dict(extensions='probtissue.nii.gz')))

IDS = ['03']

pipe = BIDSPipeline(steps=[('nucorrection', nucorrection),
                           ('tissuesegmentation', tissuesegmentation),
                           ('aggregator', aggregator)],
             pipeline_name=pipeline_name, project_path=project_path)

In [12]:
pipe.fit_transform(IDS)

/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline2/steps/nucorrection/sub-03/ses-retest/anat/sub-03_ses-retest_T1w_nu.nii.gz
/work/soft/BIDSTransformers/bidst/tests/data/ds114/sub-03/ses-retest/anat/sub-03_ses-retest_T1w.nii.gz
170901-21:09:28,268 interface INFO:
	 stdout 2017-09-01T21:09:28.268182:
170901-21:09:28,269 interface INFO:
	 stdout 2017-09-01T21:09:28.268182:Running N4 for 3-dimensional images.
170901-21:09:28,270 interface INFO:
	 stdout 2017-09-01T21:09:28.268182:
170901-21:09:28,270 interface INFO:
	 stdout 2017-09-01T21:09:28.268182:Mask not read.  Creating Otsu mask.
170901-21:09:28,772 interface INFO:
	 stdout 2017-09-01T21:09:28.772048:Current level = 1
170901-21:09:28,773 interface INFO:
	 stdout 2017-09-01T21:09:28.772048:  Iteration 1 (of 50).   Current convergence value = 0.000239112 (threshold = 0)
170901-21:09:28,773 interface INFO:
	 stdout 2017-09-01T21:09:28.772048:  Iteration 2 (of 50).   Current convergence value = 0.000239054 (t

170901-21:09:33,318 interface INFO:
	 stdout 2017-09-01T21:09:33.317889:  Iteration 50 (of 50).   Current convergence value = 0.000181417 (threshold = 0)
170901-21:09:33,319 interface INFO:
	 stdout 2017-09-01T21:09:33.317889:Current level = 2
170901-21:09:33,320 interface INFO:
	 stdout 2017-09-01T21:09:33.317889:  Iteration 1 (of 50).   Current convergence value = 0.000417663 (threshold = 0)
170901-21:09:33,321 interface INFO:
	 stdout 2017-09-01T21:09:33.317889:  Iteration 2 (of 50).   Current convergence value = 0.000412344 (threshold = 0)
170901-21:09:33,322 interface INFO:
	 stdout 2017-09-01T21:09:33.317889:  Iteration 3 (of 50).   Current convergence value = 0.000407117 (threshold = 0)
170901-21:09:33,823 interface INFO:
	 stdout 2017-09-01T21:09:33.823284:  Iteration 4 (of 50).   Current convergence value = 0.000402133 (threshold = 0)
170901-21:09:33,824 interface INFO:
	 stdout 2017-09-01T21:09:33.823284:  Iteration 5 (of 50).   Current convergence value = 0.000397216 (thresh

170901-21:09:37,866 interface INFO:
	 stdout 2017-09-01T21:09:37.863133:  Iteration 3 (of 50).   Current convergence value = 0.00101323 (threshold = 0)
170901-21:09:37,867 interface INFO:
	 stdout 2017-09-01T21:09:37.863133:  Iteration 4 (of 50).   Current convergence value = 0.000988439 (threshold = 0)
170901-21:09:38,368 interface INFO:
	 stdout 2017-09-01T21:09:38.368353:  Iteration 5 (of 50).   Current convergence value = 0.000963682 (threshold = 0)
170901-21:09:38,369 interface INFO:
	 stdout 2017-09-01T21:09:38.368353:  Iteration 6 (of 50).   Current convergence value = 0.000939664 (threshold = 0)
170901-21:09:38,370 interface INFO:
	 stdout 2017-09-01T21:09:38.368353:  Iteration 7 (of 50).   Current convergence value = 0.000916096 (threshold = 0)
170901-21:09:38,370 interface INFO:
	 stdout 2017-09-01T21:09:38.368353:  Iteration 8 (of 50).   Current convergence value = 0.000894336 (threshold = 0)
170901-21:09:38,372 interface INFO:
	 stdout 2017-09-01T21:09:38.368353:  Iteration

170901-21:09:42,916 interface INFO:
	 stdout 2017-09-01T21:09:42.916794:  Iteration 6 (of 50).   Current convergence value = 0.00144286 (threshold = 0)
170901-21:09:42,917 interface INFO:
	 stdout 2017-09-01T21:09:42.916794:  Iteration 7 (of 50).   Current convergence value = 0.00137512 (threshold = 0)
170901-21:09:42,918 interface INFO:
	 stdout 2017-09-01T21:09:42.916794:  Iteration 8 (of 50).   Current convergence value = 0.00131274 (threshold = 0)
170901-21:09:42,919 interface INFO:
	 stdout 2017-09-01T21:09:42.916794:  Iteration 9 (of 50).   Current convergence value = 0.00125571 (threshold = 0)
170901-21:09:42,920 interface INFO:
	 stdout 2017-09-01T21:09:42.916794:  Iteration 10 (of 50).   Current convergence value = 0.00120269 (threshold = 0)
170901-21:09:42,920 interface INFO:
	 stdout 2017-09-01T21:09:42.916794:  Iteration 11 (of 50).   Current convergence value = 0.00115388 (threshold = 0)
170901-21:09:43,422 interface INFO:
	 stdout 2017-09-01T21:09:43.422165:  Iteration 12

170901-21:09:46,966 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:       1: _1 (0x221eb60)
170901-21:09:46,967 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:     Required Input Names: Primary
170901-21:09:46,968 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:     NumberOfRequiredInputs: 1
170901-21:09:46,968 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:     Outputs: 
170901-21:09:46,969 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:       Primary: (0x2213f80)
170901-21:09:46,970 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:     Indexed Outputs: 
170901-21:09:46,971 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:       0: Primary (0x2213f80)
170901-21:09:46,971 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:     NumberOfRequiredOutputs: 1
170901-21:09:46,972 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:     Number Of Threads: 1
170901-21:09:46,973 interface INFO:
	 stdout 2017-09-01T21:09:46.956371:     ReleaseDataFlag: Off
170901-21

...

170901-21:09:59,726 interface INFO:
	 stdout 2017-09-01T21:09:59.723602:  Iteration 39 (of 50).   Current convergence value = 0.00022915 (threshold = 0)
170901-21:09:59,727 interface INFO:
	 stdout 2017-09-01T21:09:59.723602:  Iteration 40 (of 50).   Current convergence value = 0.000226949 (threshold = 0)
170901-21:10:00,229 interface INFO:
	 stdout 2017-09-01T21:10:00.229122:  Iteration 41 (of 50).   Current convergence value = 0.000224756 (threshold = 0)
170901-21:10:00,229 interface INFO:
	 stdout 2017-09-01T21:10:00.229122:  Iteration 42 (of 50).   Current convergence value = 0.000222693 (threshold = 0)
170901-21:10:00,230 interface INFO:
	 stdout 2017-09-01T21:10:00.229122:  Iteration 43 (of 50).   Current convergence value = 0.000220544 (threshold = 0)
170901-21:10:00,231 interface INFO:
	 stdout 2017-09-01T21:10:00.229122:  Iteration 44 (of 50).   Current convergence value = 0.000218478 (threshold = 0)
170901-21:10:00,232 interface INFO:
	 stdout 2017-09-01T21:10:00.229122:  Ite

170901-21:10:04,779 interface INFO:
	 stdout 2017-09-01T21:10:04.778520:  Iteration 42 (of 50).   Current convergence value = 0.000402595 (threshold = 0)
170901-21:10:04,780 interface INFO:
	 stdout 2017-09-01T21:10:04.778520:  Iteration 43 (of 50).   Current convergence value = 0.000395378 (threshold = 0)
170901-21:10:04,780 interface INFO:
	 stdout 2017-09-01T21:10:04.778520:  Iteration 44 (of 50).   Current convergence value = 0.000388489 (threshold = 0)
170901-21:10:04,781 interface INFO:
	 stdout 2017-09-01T21:10:04.778520:  Iteration 45 (of 50).   Current convergence value = 0.000382634 (threshold = 0)
170901-21:10:05,283 interface INFO:
	 stdout 2017-09-01T21:10:05.283433:  Iteration 46 (of 50).   Current convergence value = 0.000374775 (threshold = 0)
170901-21:10:05,284 interface INFO:
	 stdout 2017-09-01T21:10:05.283433:  Iteration 47 (of 50).   Current convergence value = 0.000368177 (threshold = 0)
170901-21:10:05,284 interface INFO:
	 stdout 2017-09-01T21:10:05.283433:  It

170901-21:10:09,827 interface INFO:
	 stdout 2017-09-01T21:10:09.827817:  Iteration 45 (of 50).   Current convergence value = 0.000538106 (threshold = 0)
170901-21:10:09,828 interface INFO:
	 stdout 2017-09-01T21:10:09.827817:  Iteration 46 (of 50).   Current convergence value = 0.000532447 (threshold = 0)
170901-21:10:09,829 interface INFO:
	 stdout 2017-09-01T21:10:09.827817:  Iteration 47 (of 50).   Current convergence value = 0.000527104 (threshold = 0)
170901-21:10:09,830 interface INFO:
	 stdout 2017-09-01T21:10:09.827817:  Iteration 48 (of 50).   Current convergence value = 0.000523141 (threshold = 0)
170901-21:10:09,830 interface INFO:
	 stdout 2017-09-01T21:10:09.827817:  Iteration 49 (of 50).   Current convergence value = 0.000518503 (threshold = 0)
170901-21:10:10,332 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:  Iteration 50 (of 50).   Current convergence value = 0.000513945 (threshold = 0)
170901-21:10:10,333 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:   N

170901-21:10:10,415 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:         BufferedRegion: 
170901-21:10:10,416 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:           Dimension: 3
170901-21:10:10,416 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:           Index: [0, 0, 0]
170901-21:10:10,417 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:           Size: [11, 11, 11]
170901-21:10:10,419 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:         RequestedRegion: 
170901-21:10:10,419 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:           Dimension: 3
170901-21:10:10,420 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:           Index: [0, 0, 0]
170901-21:10:10,421 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:           Size: [11, 11, 11]
170901-21:10:10,422 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:         Spacing: [31.5, 24.6923, 31.5]
170901-21:10:10,423 interface INFO:
	 stdout 2017-09-01T21:10:10.332214:         Origin: [32.82

CSF_volume     GM_volume     WM_volume
03 retest  3.565994e+06  1.737468e+06  1.357634e+06
   test    3.206183e+06  1.632618e+06  1.412257e+06

In [13]:
!tree -a ../bidst/tests/data/ds114/

../bidst/tests/data/ds114/
├── derivatives
│   ├── example_pipeline
│   │   └── steps
│   │       ├── nucorrection
│   │       │   └── sub-01
│   │       │       ├── ses-retest
│   │       │       │   └── anat
│   │       │       │       └── sub-01_ses-retest_T1w_nu.nii.gz
│   │       │       └── ses-test
│   │       │           └── anat
│   │       │               └── sub-01_ses-test_T1w_nu.nii.gz
│   │       ├── skullstripping
│   │       │   └── sub-01
│   │       │       ├── ses-retest
│   │       │       │   └── anat
│   │       │       │       └── sub-01_ses-retest_T1w_brain.nii.gz
│   │       │       └── ses-test
│   │       │           └── anat
│   │       │               └── sub-01_ses-test_T1w_brain.nii.gz
│   │       └── tissuesegmentation
│   │           └── sub-01
│   │               ├── ses-retest
│   │               │   └── anat
│   │               │       ├── sub-01_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
│   │               │       ├── sub-01_ses-retest_T1w_br

In [14]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'test_pipeline_with_ml'

nucorrection = NUCorrectionTransformer(gather_steps=('source', dict(extensions='T1w.nii.gz')))

skullstripping = SkullStrippingTransformer(gather_steps=('nucorrection', dict(extensions='nu.nii.gz')))

tissuesegmentation = TissueSegmentation(gather_steps=('skullstripping', dict(extensions='brain.nii.gz')))

aggregator = StatAggregator(function=get_tissue_stats, 
                            gather_steps=('tissuesegmentation', dict(extensions='probtissue.nii.gz')))

logisticregression = LogisticRegression()

IDS = ['01', '02']
target = np.array([0, 0, 1, 1])

pipe = BIDSPipeline(steps=[('nucorrection', nucorrection),
                           ('skullstripping', skullstripping),
                           ('tissuesegmentation', tissuesegmentation),
                           ('aggregator', aggregator),
                           ('logisticregression', logisticregression)],
             pipeline_name=pipeline_name, project_path=project_path)

In [24]:
pipe.pipeline_name

'test_pipeline_with_ml'

In [23]:
pipe.named_steps

{'aggregator': StatAggregator(function=<function get_tissue_stats at 0x7f59c5dedd90>,
         gather_steps=('tissuesegmentation', {'extensions': 'probtissue.nii.gz'}),
         pipeline_name='test_pipeline_with_ml',
         project_path='/work/soft/BIDSTransformers/bidst/tests/data/ds114',
         transformer_name='aggregator'),
 'logisticregression': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'nucorrection': NUCorrectionTransformer(backend='ants', backend_param={},
             gather_steps=('source', {'extensions': 'T1w.nii.gz'}),
             pipeline_name='test_pipeline_with_ml',
             project_path='/work/soft/BIDSTransformers/bidst/tests/data/ds114',
             transformer_name='nucorrection'),
 'skullstripping': SkullStrippingTransformer(backend

In [26]:
#pipe.get_params()

In [15]:
pipe.fit(X=IDS, y=target)

/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline_with_ml/steps/nucorrection/sub-01/ses-retest/anat/sub-01_ses-retest_T1w_nu.nii.gz
/work/soft/BIDSTransformers/bidst/tests/data/ds114/sub-01/ses-retest/anat/sub-01_ses-retest_T1w.nii.gz
170901-21:25:49,56 interface INFO:
	 stdout 2017-09-01T21:25:49.056383:
170901-21:25:49,57 interface INFO:
	 stdout 2017-09-01T21:25:49.056383:Running N4 for 3-dimensional images.
170901-21:25:49,58 interface INFO:
	 stdout 2017-09-01T21:25:49.056383:
170901-21:25:49,58 interface INFO:
	 stdout 2017-09-01T21:25:49.056383:Mask not read.  Creating Otsu mask.
170901-21:25:49,560 interface INFO:
	 stdout 2017-09-01T21:25:49.560551:Current level = 1
170901-21:25:49,561 interface INFO:
	 stdout 2017-09-01T21:25:49.560551:  Iteration 1 (of 50).   Current convergence value = 0.000264527 (threshold = 0)
170901-21:25:49,562 interface INFO:
	 stdout 2017-09-01T21:25:49.560551:  Iteration 2 (of 50).   Current convergence value = 0.000262299

170901-21:25:54,106 interface INFO:
	 stdout 2017-09-01T21:25:54.104418:  Iteration 50 (of 50).   Current convergence value = 0.000193532 (threshold = 0)
170901-21:25:54,107 interface INFO:
	 stdout 2017-09-01T21:25:54.104418:Current level = 2
170901-21:25:54,109 interface INFO:
	 stdout 2017-09-01T21:25:54.104418:  Iteration 1 (of 50).   Current convergence value = 0.000425829 (threshold = 0)
170901-21:25:54,610 interface INFO:
	 stdout 2017-09-01T21:25:54.610931:  Iteration 2 (of 50).   Current convergence value = 0.000419074 (threshold = 0)
170901-21:25:54,611 interface INFO:
	 stdout 2017-09-01T21:25:54.610931:  Iteration 3 (of 50).   Current convergence value = 0.000411931 (threshold = 0)
170901-21:25:54,612 interface INFO:
	 stdout 2017-09-01T21:25:54.610931:  Iteration 4 (of 50).   Current convergence value = 0.000406177 (threshold = 0)
170901-21:25:54,613 interface INFO:
	 stdout 2017-09-01T21:25:54.610931:  Iteration 5 (of 50).   Current convergence value = 0.000400012 (thresh

170901-21:25:59,159 interface INFO:
	 stdout 2017-09-01T21:25:59.155330:  Iteration 3 (of 50).   Current convergence value = 0.000954957 (threshold = 0)
170901-21:25:59,159 interface INFO:
	 stdout 2017-09-01T21:25:59.155330:  Iteration 4 (of 50).   Current convergence value = 0.000926239 (threshold = 0)
170901-21:25:59,660 interface INFO:
	 stdout 2017-09-01T21:25:59.660816:  Iteration 5 (of 50).   Current convergence value = 0.000895458 (threshold = 0)
170901-21:25:59,661 interface INFO:
	 stdout 2017-09-01T21:25:59.660816:  Iteration 6 (of 50).   Current convergence value = 0.000869874 (threshold = 0)
170901-21:25:59,662 interface INFO:
	 stdout 2017-09-01T21:25:59.660816:  Iteration 7 (of 50).   Current convergence value = 0.000843455 (threshold = 0)
170901-21:25:59,663 interface INFO:
	 stdout 2017-09-01T21:25:59.660816:  Iteration 8 (of 50).   Current convergence value = 0.000818881 (threshold = 0)
170901-21:25:59,663 interface INFO:
	 stdout 2017-09-01T21:25:59.660816:  Iteratio

170901-21:26:04,208 interface INFO:
	 stdout 2017-09-01T21:26:04.205832:  Iteration 6 (of 50).   Current convergence value = 0.00116415 (threshold = 0)
170901-21:26:04,208 interface INFO:
	 stdout 2017-09-01T21:26:04.205832:  Iteration 7 (of 50).   Current convergence value = 0.00111118 (threshold = 0)
170901-21:26:04,209 interface INFO:
	 stdout 2017-09-01T21:26:04.205832:  Iteration 8 (of 50).   Current convergence value = 0.0010646 (threshold = 0)
170901-21:26:04,710 interface INFO:
	 stdout 2017-09-01T21:26:04.710857:  Iteration 9 (of 50).   Current convergence value = 0.00102261 (threshold = 0)
170901-21:26:04,711 interface INFO:
	 stdout 2017-09-01T21:26:04.710857:  Iteration 10 (of 50).   Current convergence value = 0.000982827 (threshold = 0)
170901-21:26:04,712 interface INFO:
	 stdout 2017-09-01T21:26:04.710857:  Iteration 11 (of 50).   Current convergence value = 0.000946726 (threshold = 0)
170901-21:26:04,713 interface INFO:
	 stdout 2017-09-01T21:26:04.710857:  Iteration 1

170901-21:26:08,264 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:       1: _1 (0x2a4cb70)
170901-21:26:08,264 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:     Required Input Names: Primary
170901-21:26:08,265 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:     NumberOfRequiredInputs: 1
170901-21:26:08,266 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:     Outputs: 
170901-21:26:08,266 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:       Primary: (0x2a41f90)
170901-21:26:08,267 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:     Indexed Outputs: 
170901-21:26:08,267 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:       0: Primary (0x2a41f90)
170901-21:26:08,268 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:     NumberOfRequiredOutputs: 1
170901-21:26:08,269 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:     Number Of Threads: 1
170901-21:26:08,269 interface INFO:
	 stdout 2017-09-01T21:26:08.251473:     ReleaseDataFlag: Off
170901-21

...

170901-21:27:16,930 interface INFO:
	 stdout 2017-09-01T21:27:16.930532:  Iteration 16 (of 50).   Current convergence value = 0.00125165 (threshold = 0)
170901-21:27:16,931 interface INFO:
	 stdout 2017-09-01T21:27:16.930532:  Iteration 17 (of 50).   Current convergence value = 0.00123145 (threshold = 0)
170901-21:27:16,932 interface INFO:
	 stdout 2017-09-01T21:27:16.930532:  Iteration 18 (of 50).   Current convergence value = 0.00121491 (threshold = 0)
170901-21:27:16,932 interface INFO:
	 stdout 2017-09-01T21:27:16.930532:  Iteration 19 (of 50).   Current convergence value = 0.00119798 (threshold = 0)
170901-21:27:16,933 interface INFO:
	 stdout 2017-09-01T21:27:16.930532:  Iteration 20 (of 50).   Current convergence value = 0.00118369 (threshold = 0)
170901-21:27:17,434 interface INFO:
	 stdout 2017-09-01T21:27:17.434853:  Iteration 21 (of 50).   Current convergence value = 0.00117709 (threshold = 0)
170901-21:27:17,436 interface INFO:
	 stdout 2017-09-01T21:27:17.434853:  Iteratio

170901-21:27:19,987 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:       Reference Count: 1
170901-21:27:19,987 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:       Modified Time: 110
170901-21:27:19,988 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:       Debug: Off
170901-21:27:19,989 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:       Object Name: 
170901-21:27:19,990 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:       Observers: 
170901-21:27:19,990 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:         none
170901-21:27:19,991 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:       Thread Count: 1
170901-21:27:19,991 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:       Global Maximum Number Of Threads: 128
170901-21:27:19,992 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:       Global Default Number Of Threads: 1
170901-21:27:19,993 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:     CoordinateTolerance: 1e-06
170901-21:27:

170901-21:27:20,50 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:               none
170901-21:27:20,51 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:             Pointer: 0x20dfd80
170901-21:27:20,51 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:             Container manages memory: true
170901-21:27:20,52 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:             Size: 1331
170901-21:27:20,53 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:             Capacity: 1331
170901-21:27:20,54 interface INFO:
	 stdout 2017-09-01T21:27:19.959881:Elapsed time: 18.7236
Renaming filename: sub-01_ses-retest_T1w_brain_pveseg.nii.gz
New filename: sub-01_ses-retest_T1w_brain_pveseg.nii.gz
Renaming filename: sub-01_ses-retest_T1w_brain_pve_1.nii.gz
New filename: sub-01_ses-retest_T1w_brain_class-GM_probtissue.nii.gz
Renaming filename: sub-01_ses-retest_T1w_brain_pve_0.nii.gz
New filename: sub-01_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
Renaming filename: sub-01_se

SUBJECT 02
{'extensions': 'probtissue.nii.gz'}
[File(filename='/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline1/steps/tissuesegmentation/sub-02/ses-retest/anat/sub-02_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz', subject='02', session='retest', type='probtissue', modality='anat'), File(filename='/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline1/steps/tissuesegmentation/sub-02/ses-retest/anat/sub-02_ses-retest_T1w_brain_class-GM_probtissue.nii.gz', subject='02', session='retest', type='probtissue', modality='anat'), File(filename='/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline1/steps/tissuesegmentation/sub-02/ses-retest/anat/sub-02_ses-retest_T1w_brain_class-WM_probtissue.nii.gz', subject='02', session='retest', type='probtissue', modality='anat'), File(filename='/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline1/steps/tissuesegmentation/sub-02/ses-test/anat/sub-02_ses-

BIDSPipeline(memory=None, pipeline_name='test_pipeline_with_ml',
       project_path='/work/soft/BIDSTransformers/bidst/tests/data/ds114',
       steps=[('nucorrection', NUCorrectionTransformer(backend='ants', backend_param={},
            gather_steps=('source', {'extensions': 'T1w.nii.gz'}),
            pipeline_name='test_pipeline_with_ml',
            project_path='/work/soft/BIDSTransformers/bidst/tests/data/ds114',
            transform...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [19]:
#! tree -a ../bidst/tests/data/ds114/derivatives/test_pipeline_with_ml/

In [16]:
probs = pipe.predict_proba(['03'])

/work/soft/BIDSTransformers/bidst/tests/data/ds114/derivatives/test_pipeline_with_ml/steps/nucorrection/sub-03/ses-retest/anat/sub-03_ses-retest_T1w_nu.nii.gz
/work/soft/BIDSTransformers/bidst/tests/data/ds114/sub-03/ses-retest/anat/sub-03_ses-retest_T1w.nii.gz
170901-21:38:46,157 interface INFO:
	 stdout 2017-09-01T21:38:46.157500:
170901-21:38:46,158 interface INFO:
	 stdout 2017-09-01T21:38:46.157500:Running N4 for 3-dimensional images.
170901-21:38:46,160 interface INFO:
	 stdout 2017-09-01T21:38:46.157500:
170901-21:38:46,161 interface INFO:
	 stdout 2017-09-01T21:38:46.157500:Mask not read.  Creating Otsu mask.
170901-21:38:46,662 interface INFO:
	 stdout 2017-09-01T21:38:46.662275:Current level = 1
170901-21:38:46,663 interface INFO:
	 stdout 2017-09-01T21:38:46.662275:  Iteration 1 (of 50).   Current convergence value = 0.000239112 (threshold = 0)
170901-21:38:46,663 interface INFO:
	 stdout 2017-09-01T21:38:46.662275:  Iteration 2 (of 50).   Current convergence value = 0.00023

170901-21:38:51,210 interface INFO:
	 stdout 2017-09-01T21:38:51.209376:  Iteration 50 (of 50).   Current convergence value = 0.000181417 (threshold = 0)
170901-21:38:51,211 interface INFO:
	 stdout 2017-09-01T21:38:51.209376:Current level = 2
170901-21:38:51,212 interface INFO:
	 stdout 2017-09-01T21:38:51.209376:  Iteration 1 (of 50).   Current convergence value = 0.000417663 (threshold = 0)
170901-21:38:51,213 interface INFO:
	 stdout 2017-09-01T21:38:51.209376:  Iteration 2 (of 50).   Current convergence value = 0.000412344 (threshold = 0)
170901-21:38:51,214 interface INFO:
	 stdout 2017-09-01T21:38:51.209376:  Iteration 3 (of 50).   Current convergence value = 0.000407117 (threshold = 0)
170901-21:38:51,214 interface INFO:
	 stdout 2017-09-01T21:38:51.209376:  Iteration 4 (of 50).   Current convergence value = 0.000402133 (threshold = 0)
170901-21:38:51,716 interface INFO:
	 stdout 2017-09-01T21:38:51.716110:  Iteration 5 (of 50).   Current convergence value = 0.000397216 (thresh

170901-21:38:55,770 interface INFO:
	 stdout 2017-09-01T21:38:55.767232:  Iteration 3 (of 50).   Current convergence value = 0.00101323 (threshold = 0)
170901-21:38:55,771 interface INFO:
	 stdout 2017-09-01T21:38:55.767232:  Iteration 4 (of 50).   Current convergence value = 0.000988439 (threshold = 0)
170901-21:38:56,272 interface INFO:
	 stdout 2017-09-01T21:38:56.272394:  Iteration 5 (of 50).   Current convergence value = 0.000963682 (threshold = 0)
170901-21:38:56,273 interface INFO:
	 stdout 2017-09-01T21:38:56.272394:  Iteration 6 (of 50).   Current convergence value = 0.000939664 (threshold = 0)
170901-21:38:56,274 interface INFO:
	 stdout 2017-09-01T21:38:56.272394:  Iteration 7 (of 50).   Current convergence value = 0.000916096 (threshold = 0)
170901-21:38:56,275 interface INFO:
	 stdout 2017-09-01T21:38:56.272394:  Iteration 8 (of 50).   Current convergence value = 0.000894336 (threshold = 0)
170901-21:38:56,276 interface INFO:
	 stdout 2017-09-01T21:38:56.272394:  Iteration

170901-21:39:00,822 interface INFO:
	 stdout 2017-09-01T21:39:00.821810:  Iteration 6 (of 50).   Current convergence value = 0.00144286 (threshold = 0)
170901-21:39:00,823 interface INFO:
	 stdout 2017-09-01T21:39:00.821810:  Iteration 7 (of 50).   Current convergence value = 0.00137512 (threshold = 0)
170901-21:39:00,824 interface INFO:
	 stdout 2017-09-01T21:39:00.821810:  Iteration 8 (of 50).   Current convergence value = 0.00131274 (threshold = 0)
170901-21:39:00,824 interface INFO:
	 stdout 2017-09-01T21:39:00.821810:  Iteration 9 (of 50).   Current convergence value = 0.00125571 (threshold = 0)
170901-21:39:01,326 interface INFO:
	 stdout 2017-09-01T21:39:01.326080:  Iteration 10 (of 50).   Current convergence value = 0.00120269 (threshold = 0)
170901-21:39:01,327 interface INFO:
	 stdout 2017-09-01T21:39:01.326080:  Iteration 11 (of 50).   Current convergence value = 0.00115388 (threshold = 0)
170901-21:39:01,327 interface INFO:
	 stdout 2017-09-01T21:39:01.326080:  Iteration 12

170901-21:39:04,881 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:       1: _1 (0x305cb70)
170901-21:39:04,882 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:     Required Input Names: Primary
170901-21:39:04,882 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:     NumberOfRequiredInputs: 1
170901-21:39:04,883 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:     Outputs: 
170901-21:39:04,884 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:       Primary: (0x3051f90)
170901-21:39:04,885 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:     Indexed Outputs: 
170901-21:39:04,886 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:       0: Primary (0x3051f90)
170901-21:39:04,886 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:     NumberOfRequiredOutputs: 1
170901-21:39:04,887 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:     Number Of Threads: 1
170901-21:39:04,888 interface INFO:
	 stdout 2017-09-01T21:39:04.863528:     ReleaseDataFlag: Off
170901-21

...

170901-21:39:13,126 interface INFO:
	 stdout 2017-09-01T21:39:13.122778:  Iteration 36 (of 50).   Current convergence value = 0.000204223 (threshold = 0)
170901-21:39:13,628 interface INFO:
	 stdout 2017-09-01T21:39:13.628213:  Iteration 37 (of 50).   Current convergence value = 0.000203063 (threshold = 0)
170901-21:39:13,629 interface INFO:
	 stdout 2017-09-01T21:39:13.628213:  Iteration 38 (of 50).   Current convergence value = 0.000202812 (threshold = 0)
170901-21:39:13,629 interface INFO:
	 stdout 2017-09-01T21:39:13.628213:  Iteration 39 (of 50).   Current convergence value = 0.000201003 (threshold = 0)
170901-21:39:13,630 interface INFO:
	 stdout 2017-09-01T21:39:13.628213:  Iteration 40 (of 50).   Current convergence value = 0.00020002 (threshold = 0)
170901-21:39:13,631 interface INFO:
	 stdout 2017-09-01T21:39:13.628213:  Iteration 41 (of 50).   Current convergence value = 0.000198929 (threshold = 0)
170901-21:39:14,132 interface INFO:
	 stdout 2017-09-01T21:39:14.132709:  Ite

170901-21:39:18,182 interface INFO:
	 stdout 2017-09-01T21:39:18.179601:  Iteration 39 (of 50).   Current convergence value = 0.00022915 (threshold = 0)
170901-21:39:18,183 interface INFO:
	 stdout 2017-09-01T21:39:18.179601:  Iteration 40 (of 50).   Current convergence value = 0.000226949 (threshold = 0)
170901-21:39:18,685 interface INFO:
	 stdout 2017-09-01T21:39:18.685063:  Iteration 41 (of 50).   Current convergence value = 0.000224756 (threshold = 0)
170901-21:39:18,686 interface INFO:
	 stdout 2017-09-01T21:39:18.685063:  Iteration 42 (of 50).   Current convergence value = 0.000222693 (threshold = 0)
170901-21:39:18,686 interface INFO:
	 stdout 2017-09-01T21:39:18.685063:  Iteration 43 (of 50).   Current convergence value = 0.000220544 (threshold = 0)
170901-21:39:18,687 interface INFO:
	 stdout 2017-09-01T21:39:18.685063:  Iteration 44 (of 50).   Current convergence value = 0.000218478 (threshold = 0)
170901-21:39:18,688 interface INFO:
	 stdout 2017-09-01T21:39:18.685063:  Ite

170901-21:39:23,238 interface INFO:
	 stdout 2017-09-01T21:39:23.237801:  Iteration 42 (of 50).   Current convergence value = 0.000402595 (threshold = 0)
170901-21:39:23,239 interface INFO:
	 stdout 2017-09-01T21:39:23.237801:  Iteration 43 (of 50).   Current convergence value = 0.000395378 (threshold = 0)
170901-21:39:23,240 interface INFO:
	 stdout 2017-09-01T21:39:23.237801:  Iteration 44 (of 50).   Current convergence value = 0.000388489 (threshold = 0)
170901-21:39:23,245 interface INFO:
	 stdout 2017-09-01T21:39:23.237801:  Iteration 45 (of 50).   Current convergence value = 0.000382634 (threshold = 0)
170901-21:39:23,747 interface INFO:
	 stdout 2017-09-01T21:39:23.747112:  Iteration 46 (of 50).   Current convergence value = 0.000374775 (threshold = 0)
170901-21:39:23,748 interface INFO:
	 stdout 2017-09-01T21:39:23.747112:  Iteration 47 (of 50).   Current convergence value = 0.000368177 (threshold = 0)
170901-21:39:23,748 interface INFO:
	 stdout 2017-09-01T21:39:23.747112:  It

170901-21:39:28,295 interface INFO:
	 stdout 2017-09-01T21:39:28.295654:  Iteration 45 (of 50).   Current convergence value = 0.000538106 (threshold = 0)
170901-21:39:28,296 interface INFO:
	 stdout 2017-09-01T21:39:28.295654:  Iteration 46 (of 50).   Current convergence value = 0.000532447 (threshold = 0)
170901-21:39:28,297 interface INFO:
	 stdout 2017-09-01T21:39:28.295654:  Iteration 47 (of 50).   Current convergence value = 0.000527104 (threshold = 0)
170901-21:39:28,298 interface INFO:
	 stdout 2017-09-01T21:39:28.295654:  Iteration 48 (of 50).   Current convergence value = 0.000523141 (threshold = 0)
170901-21:39:28,298 interface INFO:
	 stdout 2017-09-01T21:39:28.295654:  Iteration 49 (of 50).   Current convergence value = 0.000518503 (threshold = 0)
170901-21:39:28,800 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:  Iteration 50 (of 50).   Current convergence value = 0.000513945 (threshold = 0)
170901-21:39:28,801 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:   N

170901-21:39:28,856 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:         BufferedRegion: 
170901-21:39:28,857 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:           Dimension: 3
170901-21:39:28,857 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:           Index: [0, 0, 0]
170901-21:39:28,858 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:           Size: [11, 11, 11]
170901-21:39:28,858 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:         RequestedRegion: 
170901-21:39:28,859 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:           Dimension: 3
170901-21:39:28,860 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:           Index: [0, 0, 0]
170901-21:39:28,860 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:           Size: [11, 11, 11]
170901-21:39:28,861 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:         Spacing: [31.5, 24.6923, 31.5]
170901-21:39:28,862 interface INFO:
	 stdout 2017-09-01T21:39:28.799960:         Origin: [32.82

In [20]:
probs

array([[  1.00000000e+000,   2.04588935e-292],
       [  1.00000000e+000,   1.45423301e-241]])

In [21]:
pipe.named_steps

{'aggregator': StatAggregator(function=<function get_tissue_stats at 0x7f59c5dedd90>,
         gather_steps=('tissuesegmentation', {'extensions': 'probtissue.nii.gz'}),
         pipeline_name='test_pipeline_with_ml',
         project_path='/work/soft/BIDSTransformers/bidst/tests/data/ds114',
         transformer_name='aggregator'),
 'logisticregression': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'nucorrection': NUCorrectionTransformer(backend='ants', backend_param={},
             gather_steps=('source', {'extensions': 'T1w.nii.gz'}),
             pipeline_name='test_pipeline_with_ml',
             project_path='/work/soft/BIDSTransformers/bidst/tests/data/ds114',
             transformer_name='nucorrection'),
 'skullstripping': SkullStrippingTransformer(backend